# training and evaluating the CPA

In [ ]:
import sys
sys.path.append('..')

import wandb
wandb.login()

from chemCPA.data import DataModule
from chemCPA.model import ComPert
import torch
import numpy as np
import time
import yaml
import lightning as L
from lightning.pytorch import seed_everything
from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

## load configuration

In [2]:
with open('../chemCPA//train_data/config_hparam.yaml') as file:
    config_data = yaml.safe_load(file)

#seed_everything()

## initialize the dataset module

In [3]:
dm = DataModule(config_data['model']['hparams']['batch_size'],
                        config_data['model']['training_hparams']['full_eval_during_train'],
                        **config_data['dataset']['data_params'])

In [ ]:
print(len(dm.datasets['training']))
print(len(dm.datasets['training_control']))
print(len(dm.datasets['training_treated']))
print(len(dm.datasets['test']))
print(len(dm.datasets['test_control']))
print(len(dm.datasets['test_treated']))
print(len(dm.datasets['ood']))

In [ ]:
print(dm.datasets['training'].num_covariates)
print(dm.datasets['training'].num_drugs)
print(dm.datasets['training'].num_knockouts)

## initialize the model

In [6]:
# initialize the model
# the knockouts embeddings are initialized as random 
model = ComPert(
    dm.datasets['training'].num_genes,
    dm.datasets['training'].num_drugs,
    dm.datasets['training'].num_knockouts,
    dm.datasets['training'].num_covariates,
    config_data['model']['hparams'],
    config_data['model']['training_hparams'],
    config_data['model']['test_hparams'],
    **config_data['model']['additional_params'],
    drug_embedding_dimension=dm.datasets['training'].drug_embedding_dimension,
    knockout_embedding_dimension=dm.datasets['training'].knockout_embedding_dimension
    )
    

## Initialize the trainer

In [ ]:
early_stop_callback = EarlyStopping('average_r2_score', 
                                    patience=model.hparams.training_hparams['patience'], 
                                    mode='max')

if (not model.hparams.training_hparams['run_eval_disentangle']) and (not model.hparams.test_hparams['run_eval_disentangle']):
    trainer = L.Trainer(
        logger=WandbLogger(log_model="all"),
        max_epochs=config_data['model']['training_hparams']['num_epochs'],
        max_time=config_data['model']['training_hparams']['max_time'],
        check_val_every_n_epoch= config_data['model']['training_hparams']['checkpoint_freq'],
        default_root_dir=config_data['model']['training_hparams']['save_dir'],
        profiler="advanced",
        callbacks=[early_stop_callback],
        #inference_mode=False
    )
else: 
    trainer = L.Trainer(
        logger=WandbLogger(log_model="all"),
        max_epochs=config_data['model']['training_hparams']['num_epochs'],
        max_time=config_data['model']['training_hparams']['max_time'],
        check_val_every_n_epoch= config_data['model']['training_hparams']['checkpoint_freq'],
        default_root_dir=config_data['model']['training_hparams']['save_dir'],
        profiler="advanced",
        callbacks=[early_stop_callback],
        inference_mode=False
    )

## train the model

In [ ]:
trainer.fit(model, datamodule=dm)

## test the model

In [ ]:
trainer.test(model, datamodule=dm)

## reload the model from check point

In [ ]:
#reload the model from check point
#model = ComPert.load_from_checkpoint('lightning_logs/yznmw0w3/checkpoints/epoch=0-step=704.ckpt')